# Stock purchase recommendations with Machine Learning

In [13]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt

In [14]:
# read in the Quandl.com API key - saved in separate file to keep it private

filepath = 'confidential-API-key.txt'  
with open(filepath) as fp:  
   my_Quandl_API_key = fp.readline()

In [15]:
start = '2015-04-22'
end = '2017-04-22'

symbol = ['AAPL']

max_holding = 100
price = web.DataReader(name=symbol, data_source='quandl', start=start, end=end, access_key=my_Quandl_API_key)
#price = price.iloc[::-1]
#price = price.dropna()
#close = price['AdjClose'].values
#up, mid, low = BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
#rsi = RSI(close, timeperiod=14)
#print("RSI (first 10 elements)\n", rsi[14:24])

In [16]:
price.head()

Attributes,Open,High,Low,Close,Volume,ExDividend,SplitRatio,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjVolume
Symbols,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,,,,,,,
2017-04-21,142.44,142.68,141.85,142.27,17320928.0,0.0,1.0,141.286216,141.524272,140.700995,141.117593,17320928.0
2017-04-20,141.22,142.92,141.16,142.44,23319562.0,0.0,1.0,140.076098,141.762328,140.016584,141.286216,23319562.0
2017-04-19,141.88,142.00,140.45,140.68,17328375.0,0.0,1.0,140.730752,140.849780,139.312335,139.540472,17328375.0
2017-04-18,141.41,142.04,141.11,141.20,14697544.0,0.0,1.0,140.264559,140.889456,139.966989,140.056260,14697544.0
2017-04-17,141.48,141.88,140.87,141.83,16582094.0,0.0,1.0,140.333992,140.730752,139.728933,140.681157,16582094.0


In [17]:
# need ascending index for the TA-lib indicators to work properly
price.sort_index(inplace=True)
price = price[['AdjVolume', 'AdjOpen', 'AdjHigh', 'AdjLow', 'AdjClose']]
price.columns = price.columns.droplevel(1)
price.head()

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose
Date,,,,,
2015-04-22,37654505.0,121.113778,122.906785,120.474781,122.668353
2015-04-23,45770902.0,122.363160,124.385061,122.210564,123.669766
2015-04-24,44525905.0,124.451822,124.585344,123.250126,124.251540
2015-04-27,96954207.0,126.187605,126.969661,125.081282,126.511872
2015-04-28,118923970.0,128.233349,128.314416,123.574394,124.518583


In [18]:
price['AdjClose'].head()

Date
2015-04-22    122.668353
2015-04-23    123.669766
2015-04-24    124.251540
2015-04-27    126.511872
2015-04-28    124.518583
Name: AdjClose, dtype: float64

In [19]:
# add row index (highest number is most recent date) - this will be used to later restack the rows for feature matrix
price['row_index'] = range(0, price.shape[0])

In [20]:
price.head()

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index
Date,,,,,,
2015-04-22,37654505.0,121.113778,122.906785,120.474781,122.668353,0
2015-04-23,45770902.0,122.363160,124.385061,122.210564,123.669766,1
2015-04-24,44525905.0,124.451822,124.585344,123.250126,124.251540,2
2015-04-27,96954207.0,126.187605,126.969661,125.081282,126.511872,3
2015-04-28,118923970.0,128.233349,128.314416,123.574394,124.518583,4


In [21]:
import talib as ta

In [23]:
def ApplyIndicator(df, attribute, indicator_func, *args):
    func_name = indicator_func.__name__
    df.loc[:, idx[func_name]] = indicator_func(df.loc[:, iloc[attribute]].values, *args)
    return df, func_name

new_df, indicator_name = ApplyIndicator(price, 'AdjClose', ta.RSI, 14)

new_df.tail()

NameError: name 'iloc' is not defined

In [220]:
new_df.head()
#confirms NaN for RSI on top for first dates

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index,RSI
Date,,,,,,,
2015-04-22,37654505.0,121.113778,122.906785,120.474781,122.668353,0,NaN
2015-04-23,45770902.0,122.363160,124.385061,122.210564,123.669766,1,NaN
2015-04-24,44525905.0,124.451822,124.585344,123.250126,124.251540,2,NaN
2015-04-27,96954207.0,126.187605,126.969661,125.081282,126.511872,3,NaN
2015-04-28,118923970.0,128.233349,128.314416,123.574394,124.518583,4,NaN


## define the goal metric column - swap this out with whatever method desired
eg: store in row N if on day N+1 the stock had at least a 0.5% increase of from Open to Close
important to make sure that there is no row offset issues that include a look into the future

In [234]:
# trade strategy: after market close on day N, set a buy at market open on day N+1 and sell at market close on day N+1
# for expected gain of at least 0.5%

# use helper column to calculate profit
new_df['strategy_profit'] = (new_df.AdjClose.shift(-1) - new_df.AdjOpen.shift(-1)) / new_df.AdjOpen.shift(-1)

# use categorical field to encode "setup_for_succesful_trade": 1=yes, 0=no
new_df['setup_for_sucessful_trade'] = new_df['strategy_profit'] > 0.005
new_df.tail()

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index,RSI,setup_for_sucessful_trade,strategy_profit
Date,,,,,,,,,
2017-04-17,16582094.0,140.333992,140.730752,139.728933,140.681157,500,54.018130,False,-0.001485
2017-04-18,14697544.0,140.264559,140.889456,139.966989,140.056260,501,50.438090,False,-0.008458
2017-04-19,17328375.0,140.730752,140.849780,139.312335,139.540472,502,47.632042,True,0.008639
2017-04-20,23319562.0,140.076098,141.762328,140.016584,141.286216,503,56.460990,False,-0.001193
2017-04-21,17320928.0,141.286216,141.524272,140.700995,141.117593,504,55.487879,False,NaN


In [237]:
new_df.setup_for_sucessful_trade.sum()

167

In [267]:
n_days_features = 5
n_data_points = new_df.shape[0]

In [304]:
Xy = pd.DataFrame()

targetDFrowCounter = 0

for i in range(n_data_points-1, n_days_features-1, -1):
    
    Xy.loc[targetDFrowCounter, 'setup_for_sucessful_trade'] = new_df.iloc[i,:]['setup_for_sucessful_trade']
    
    #print(new_df.iloc[i-n_days_features+1:i+1,:]['setup_for_sucessful_trade']) # just checking for right dates/rows
    print(type(new_df.iloc[i-n_days_features+1:i+1,:]['setup_for_sucessful_trade'].values))
    Xy.iloc[targetDFrowCounter,0:6] = [new_df.iloc[i-n_days_features+1:i+1,:]['AdjOpen'].values]
    
    targetDFrowCounter = targetDFrowCounter + 1

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'nump

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'nump

In [306]:
Xy
# issue: need to invert the order, get column names, do the application for all input columns

,setup_for_sucessful_trade
0,"[140.33399191137, 140.26455892131, 140.7307518..."
1,"[140.76050885032, 140.33399191137, 140.2645589..."
2,"[140.45301989433, 140.76050885032, 140.3339919..."
3,"[141.78216570407, 140.45301989433, 140.7605088..."
4,"[142.43681961036, 141.78216570407, 140.4530198..."
5,"[142.5657665919, 142.43681961036, 141.78216570..."
6,"[143.12123051238, 142.5657665919, 142.43681961..."
7,"[143.05179752232002, 143.12123051238, 142.5657..."
8,"[142.08965466005, 143.05179752232002, 143.1212..."
9,"[142.54592859474, 142.08965466005, 143.0517975..."


### indexing testing around: works for one column:

In [221]:
price.loc[:,('Open','AAPL')].tail(20)

KeyError: u"None of [('Open', 'AAPL')] are in the [columns]"

In [104]:
price['Open']['AAPL'].values[-20:]

array([129.38, 129.08, 129.46, 129.  , 128.75, 129.39, 129.5 , 129.14,
       129.89, 130.31, 130.02, 129.95, 130.62, 131.31, 131.  , 131.25,
       132.01, 132.64, 132.65, 133.55])

In [117]:
ta.RSI(price['Open']['AAPL'].values, 14)[-20:]

array([64.67918491, 61.52503529, 63.92482519, 59.11784324, 56.62564291,
       61.14182474, 61.87655495, 58.01067873, 63.17286007, 65.71484641,
       62.50662269, 61.72331015, 66.10200343, 69.91852913, 66.30652914,
       67.75331269, 71.72775641, 74.53015083, 74.57323366, 78.15471567])

In [129]:
price.loc[:, idx[('RSI'), ('AAPL')]] = ta.RSI(price.loc[:, idx[('Open'), ('AAPL')]].values, 14)
price['RSI']['AAPL'].tail(20)

Date
2017-03-24    64.679185
2017-03-27    61.525035
2017-03-28    63.924825
2017-03-29    59.117843
2017-03-30    56.625643
2017-03-31    61.141825
2017-04-03    61.876555
2017-04-04    58.010679
2017-04-05    63.172860
2017-04-06    65.714846
2017-04-07    62.506623
2017-04-10    61.723310
2017-04-11    66.102003
2017-04-12    69.918529
2017-04-13    66.306529
2017-04-17    67.753313
2017-04-18    71.727756
2017-04-19    74.530151
2017-04-20    74.573234
2017-04-21    78.154716
Name: AAPL, dtype: float64

In [116]:
idx = pd.IndexSlice
price.loc[:, idx[:, ('AAPL','MSFT')]].head()

Attributes   Open          High            Low          Close         \
Symbols      AAPL   MSFT   AAPL   MSFT    AAPL    MSFT   AAPL   MSFT   
Date                                                                   
2015-04-22  97.00  97.00  99.35  99.35  96.240  96.240  97.84  97.84   
2015-04-23  97.44  97.44  97.52  97.52  96.555  96.555  97.00  97.00   
2015-04-24  96.99  96.99  99.08  99.08  96.840  96.840  98.74  98.74   
2015-04-27  98.74  98.74  98.94  98.94  96.260  96.260  96.44  96.44   
2015-04-28  96.27  96.27  96.89  96.89  95.780  95.780  96.83  96.83   

Attributes      Volume                 ...        AdjOpen             \
Symbols           AAPL        MSFT     ...           AAPL       MSFT   
Date                                   ...                             
2015-04-22  19253001.0  19253001.0     ...      89.985614  89.985614   
2015-04-23   6317607.0   6317607.0     ...      90.393796  90.393796   
2015-04-24   7741339.0   7741339.0     ...      89.976337  89.976337   
2015-04-27   7203201.0   7203201.0     ...      91.599789  91.599789   
2015-04-28   4357219.0   4357219.0     ...      89.308403  89.308403   

Attributes    AdjHigh                AdjLow              AdjClose             \
Symbols          AAPL       MSFT       AAPL       MSFT       AAPL       MSFT   
Date                                                                           
2015-04-22  92.165678  92.165678  89.280572  89.280572  90.764871  90.764871   
2015-04-23  90.468011  90.468011  89.572793  89.572793  89.985614  89.985614   
2015-04-24  91.915202  91.915202  89.837184  89.837184  91.599789  91.599789   
2015-04-27  91.785326  91.785326  89.299126  89.299126  89.466109  89.466109   
2015-04-28  89.883568  89.883568  88.853836  88.853836  89.827907  89.827907   

Attributes   AdjVolume              
Symbols           AAPL        MSFT  
Date                                
2015-04-22  19253001.0  19253001.0  
2015-04-23   6317607.0   6317607.0  
2015-04-24   7741339.0   7741339.0  
2015-04-27   7203201.0   7203201.0  
2015-04-28   4357219.0   4357219.0  

[5 rows x 24 columns]

In [80]:
price.loc[:, idx[('AdjOpen','AdjClose'), ('AAPL','MSFT')]].head()

Attributes    AdjOpen              AdjClose           
Symbols          AAPL       MSFT       AAPL       MSFT
Date                                                  
2015-04-22  89.985614  89.985614  90.764871  90.764871
2015-04-23  90.393796  90.393796  89.985614  89.985614
2015-04-24  89.976337  89.976337  91.599789  91.599789
2015-04-27  91.599789  91.599789  89.466109  89.466109
2015-04-28  89.308403  89.308403  89.827907  89.827907

In [81]:
price.loc[:, idx[('AdjClose'), ('AAPL')]].head()

Date
2015-04-22    90.764871
2015-04-23    89.985614
2015-04-24    91.599789
2015-04-27    89.466109
2015-04-28    89.827907
Name: (AdjClose, AAPL), dtype: float64